# Text to Fashion Images

## 1. Upload data to S3
Here I use pokeman dataset as an example, which is composed of 833 image-text pairs. To scale up, you can just process your data into the same format.

In [1]:
import sagemaker
import boto3
import datetime
import json
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

prefix_train = 'datasets/heguan_reformed'
inputs_train = 's3://sagemaker-us-west-2-452145973879/datasets/heguan_reformed/'

# local_data_path = "example_data"
# local_data_path = "../data/heguan-reformed"
# inputs_train = sagemaker_session.upload_data(path=local_data_path, key_prefix=prefix_train)
# print(inputs_train)

# prefix_test = 'datasets/brushnet_test'
# inputs_train = sagemaker_session.upload_data(path = "test_data", key_prefix=prefix_train)
# print(inputs_train)

## 2. Upload checkpoints to S3

In [3]:
# prefix_base_model = 'checkpoints/realisticVisionV60B1_v51VAE'
# inputs_base_model = sagemaker_session.upload_data(path="ckpt/realisticVisionV60B1_v51VAE", key_prefix=prefix_base_model)
# print(inputs_base_model)

In [4]:
# prefix_base_model = 'checkpoints/sd15_urbanicv2'
# inputs_base_model = sagemaker_session.upload_data(path="../data/ckpt/sd15_urbanicv2", key_prefix=prefix_base_model)
# print(inputs_base_model)

In [5]:
# prefix_brushnet_model = 'checkpoints/random_mask_brushnet_ckpt'
# inputs_brushnet_model = sagemaker_session.upload_data(path="ckpt/random_mask_brushnet_ckpt", key_prefix=prefix_brushnet_model)
# print(inputs_brushnet_model)

## 2. Start a training job

In [ ]:
import time
from sagemaker.estimator import Estimator

region = sagemaker_session.boto_session.region_name

image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:1.13.1-gpu-py39-cu117-ubuntu20.04-sagemaker'

instance_count = 1
# instance_type = 'ml.g5.12xlarge'
instance_type = 'ml.p4d.24xlarge' ## p4d - 8*40G / p4de - 8*80G
# instance_type = 'ml.g5.48xlarge'

#     'TRAIN_DIR': '/opt/ml/input/data/train'
environment = {
    'NODE_NUMBER': str(instance_count),
    'TRAIN_DATA_PATH': f's3://{bucket}/{prefix_train}/',
#    'TEST_DATA_PATH': f's3://{bucket}/{prefix_test}/',
    'BASE_MODEL_S3_PATH': f's3://{bucket}/checkpoints/sd15_urbanicv2/', # realisticVisionV60B1_v51VAE/',  # 
    'BRUSHNET_MODEL_S3_PATH': f's3://{bucket}/checkpoints/random_mask_brushnet_ckpt/', 
    'OUTPUT_MODEL_S3_PATH': f's3://{bucket}/checkpoints/BrushNet_urbanic_random_custom_resume60k/', # destination
    'LATEST_CHECKPOINT_S3_PATH': f's3://{bucket}/checkpoints/BrushNet_urbanic_random_custommask/checkpoint-60000/'# f's3://{bucket}/checkpoints/BrushNet_ml.p4d.24xlarge_1_urbanicv2/checkpoint-20000/'
#     'OUTPUT_DIR': '/opt/ml/model'
}

estimator = Estimator(role=role,
                      entry_point='entry.py',
                      source_dir='./sm_scripts',
                      base_job_name='brushnet-launch',
                      instance_count=instance_count,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      max_run=3*24*3600, #任务最大存续时间，默认2day，需要提交ticket提升quota最大28天
                      disable_profiler=True,
                      debugger_hook_config=False)

estimator.fit({'train': inputs_train})

INFO:sagemaker:Creating training-job with name: brushnet-launch-2024-04-26-23-18-48-123


2024-04-26 23:20:45 Starting - Starting the training job...
2024-04-26 23:20:53 Pending - Training job waiting for capacity...
2024-04-26 23:21:18 Pending - Preparing the instances for training........................
2024-04-26 23:25:43 Downloading - Downloading input data............
2024-04-26 23:27:29 Downloading - Downloading the training image...